In [1]:
# Import Required Libraries
import backtrader as bt
import backtrader.feeds as btfeeds
import pandas as pd
import datetime
import numpy as np
import os

In [2]:
long_conditions = {
        'bullish_reversal_patterns': [
            'CDLHAMMER',              # Hammer
            'CDLINVERTEDHAMMER',      # Inverted Hammer
            'CDLMORNINGSTAR',         # Morning Star
            'CDLMORNINGDOJISTAR',     # Morning Doji Star
            'CDLENGULFING',           # Bullish Engulfing
            'CDLPIERCING',            # Piercing Pattern
            'CDLHARAMI',              # Bullish Harami
            'CDLHARAMICROSS',         # Bullish Harami Cross
            'CDLTAKURI',              # Takuri (Dragonfly Doji)
        ],
        
        'bullish_continuation_patterns': [
            'CDL3WHITESOLDIERS',      # Three White Soldiers
            'CDLRISEFALL3METHODS',    # Rising Three Methods
            'CDLMATHOLD',             # Mat Hold
            'CDLSEPARATINGLINES',     # Bullish Separating Lines
            'CDLTASUKIGAP',           # Bullish Tasuki Gap uptrend
        ],
        
        'bullish_bottom_patterns': [
            'CDLABANDONEDBABY',       # Abandoned Baby
            'CDLLADDERBOTTOM',        # Ladder Bottom
            'CDLMATCHINGLOW',         # Matching Low
            'CDLUNIQUE3RIVER',        # Unique Three River
        ],
        
        'bullish_special_patterns': [
            'CDL3INSIDE',             # Three Inside Up
            'CDL3OUTSIDE',            # Three Outside Up
            'CDLBELTHOLD',            # Belt Hold
            'CDLBREAKAWAY',           # Breakaway
            'CDLKICKING',             # Kicking
            'CDLKICKINGBYLENGTH',     # Kicking By Length
            'CDLSTICKSANDWICH',       # Stick Sandwich
        ]
    }

In [3]:
sell_conditions = {
        'bearish_reversal_patterns': [
            'CDLSHOOTINGSTAR',        # Shooting Star
            'CDLHANGINGMAN',          # Hanging Man
            'CDLEVENINGSTAR',         # Evening Star
            'CDLEVENINGDOJISTAR',     # Evening Doji Star
            'CDLENGULFING',           # Bearish Engulfing (negative signal)
            'CDLDARKCLOUDCOVER',      # Dark Cloud Cover
            'CDLGRAVESTONEDOJI',      # Gravestone Doji
            'CDLHARAMI',              # Bearish Harami (negative signal)
            'CDLHARAMICROSS',         # Bearish Harami Cross (negative signal)
        ],
        
        'bearish_continuation_patterns': [
            'CDL3BLACKCROWS',         # Three Black Crows
            'CDLFALL3METHODS',        # Falling Three Methods
            'CDLIDENTICAL3CROWS',     # Identical Three Crows
            'CDLADVANCEBLOCK',        # Advance Block
            'CDLSTALLEDPATTERN',      # Stalled Pattern
        ],
        
        'bearish_top_patterns': [
            'CDL2CROWS',              # Two Crows
            'CDLUPSIDEGAP2CROWS',     # Upside Gap Two Crows
            'CDLXSIDEGAP3METHODS',    # Upside/Downside Gap Three Methods
        ],
        
        'bearish_special_patterns': [
            'CDL3INSIDE',             # Three Inside Down
            'CDL3OUTSIDE',            # Three Outside Down
            'CDL3LINESTRIKE',         # Three Line Strike (bearish context)
            'CDLINNECK',              # In-Neck Line
            'CDLONNECK',              # On-Neck Line
            'CDLTHRUSTING',           # Thrusting Pattern
            'CDLBELTHOLD',            # Belt Hold (bearish context)
            'CDLBREAKAWAY',           # Breakaway (bearish context)
        ]
    }

In [4]:
# Strategy Candlestick Patterns - Practical Implementation

class Long_Candlesticks(bt.Strategy):
    """
    Strategy focusing on long candlestick patterns for trade entries
    Entry signals based on bullish patterns, iterated through dictionary long_conditions
    
    Stop-loss and take-profit based on ATR

    Ideally suited for swing trading with signals on 1H, 4H charts

    """
    
    params = (
        ('printlog', False),
        ('atr_period', 14),
        ('atr_sl_multiplier', 2.0),
        ('atr_tp_multiplier', 3.0),
    )

    def __init__(self):

        # Iterate through long_conditions dictionary to create candle pattern indicators
        self.candle_patterns = {}
        for pattern in long_conditions['bullish_reversal_patterns'] + \
                       long_conditions['bullish_continuation_patterns'] + \
                       long_conditions['bullish_bottom_patterns'] + \
                       long_conditions['bullish_special_patterns']: 
            self.candle_patterns[pattern] = getattr(bt.indicators, pattern)(self.data)

        # ATR for stop-loss and take-profit calculations
        self.atr = bt.indicators.ATR(self.data, period=self.p.atr_period)

        self.stop_loss = None
        self.take_profit = None
        
        self.order = None
        self.trade_count = 0

    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}')
            else:
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}')
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.trade_count += 1
        self.log(f'TRADE #{self.trade_count}: PROFIT {trade.pnl:.2f}', doprint=True)

    def next(self):
        if len(self.data) < 60:  # Wait for indicators to stabilize
            return

        if self.order:
            return

        if not self.position:
            # Check for any bullish candlestick pattern to enter long
            try:
                if any(self.candle_patterns[pattern][0] != 0 for pattern in self.candle_patterns):
                    self.log('BUY SIGNAL: Bullish candlestick pattern detected')
                    self.order = self.buy()
                    
                    # Set stop-loss and take-profit based on ATR
                    self.stop_loss = self.data.close[0] - (self.atr[0] * self.p.atr_sl_multiplier)
                    self.take_profit = self.data.close[0] + (self.atr[0] * self.p.atr_tp_multiplier)
                    self.log(f'SET SL: {self.stop_loss:.2f}, TP: {self.take_profit:.2f}')
                    
            except (IndexError, TypeError):
                return
        else:
            # Manage existing position with ATR-based stop-loss and take-profit
            if self.data.close[0] <= self.stop_loss:
                self.log('STOP-LOSS HIT')
                self.order = self.sell()
            elif self.data.close[0] >= self.take_profit:
                self.log('TAKE-PROFIT HIT')
                self.order = self.sell()
                return

    def stop(self):
        self.log(f'Strategy Simplified: {self.trade_count} trades, Final Value: ${self.broker.getvalue():.2f}', doprint=True)

print("✓ Strategy000Simplified created - practical implementation with realistic exit levels!")

✓ Strategy000Simplified created - practical implementation with realistic exit levels!


In [ ]:
# Backtest Function for Strategy000Simplified

def test_simplified_strategy():
    """Test the simplified strategy with S&P 500 data"""
    print("=== TESTING STRATEGY000SIMPLIFIED ===\n")
    
    cerebro = bt.Cerebro()
    
    # Load S&P 500 data
    spy_file = r'../04_S&P500_quant_analysis/01_data/S&P500_D_1789-05-01_2025-09-17.csv'
    df = pd.read_csv(spy_file, skiprows=3, header=None)
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').dropna()
    
    # Use last 5 years for testing
    end_date = df['Date'].max()
    start_date = end_date - pd.DateOffset(years=5)
    df_recent = df[df['Date'] >= start_date].copy()
    
    print(f"📊 Data: {len(df_recent)} trading days from {start_date.date()} to {end_date.date()}")
    
    # Create temporary file for backtrader
    temp_file = 'temp_simplified_test.csv'
    df_recent.to_csv(temp_file, index=False)
    
    # Create data feed
    data = btfeeds.GenericCSVData(
        dataname=temp_file,
        dtformat=('%Y-%m-%d'),
        datetime=0, open=1, high=2, low=3, close=4, volume=5,
        openinterest=-1, headers=True,
    )

    # Add data and strategy
    cerebro.adddata(data)
    cerebro.addstrategy(Long_Candlesticks, printlog=False)
    
    # Set initial capital and commission
    initial_capital = 100000.0
    cerebro.broker.setcash(initial_capital)
    cerebro.broker.setcommission(commission=0.001)  # 0.1% commission
    
    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")

    print(f"💰 Starting Capital: ${initial_capital:,.2f}")
    print("🚀 Running backtest...\n")
    
    # Run backtest
    results = cerebro.run()
    
    # Clean up temporary file
    if os.path.exists(temp_file):
        os.remove(temp_file)
    
    # Calculate results
    final_value = cerebro.broker.getvalue()
    total_return = ((final_value - initial_capital) / initial_capital) * 100
    
    print(f"\n=== BACKTEST RESULTS ===")
    print(f"💰 Final Portfolio Value: ${final_value:,.2f}")
    print(f"📈 Total Return: {total_return:.2f}%")
    print(f"💵 Net Profit: ${final_value - initial_capital:,.2f}")
    
    # Analyze trades
    strat = results[0]
    trade_analysis = strat.analyzers.trades.get_analysis()
    
    if trade_analysis and hasattr(trade_analysis, 'total'):
        total_trades = getattr(trade_analysis.total, 'total', 0)
        print(f"\n=== TRADE ANALYSIS ===")
        print(f"📊 Total Trades: {total_trades}")
        
        # Winning trades
        if hasattr(trade_analysis, 'won'):
            won_trades = getattr(trade_analysis.won, 'total', 0)
            won_pnl = getattr(trade_analysis.won, 'pnl', {}).get('total', 0)
            win_rate = (won_trades / total_trades * 100) if total_trades > 0 else 0
            print(f"✅ Winning Trades: {won_trades} ({win_rate:.1f}%)")
            if won_pnl:
                print(f"💰 Winning PnL: ${won_pnl:.2f}")
        
        # Losing trades  
        if hasattr(trade_analysis, 'lost'):
            lost_trades = getattr(trade_analysis.lost, 'total', 0)
            lost_pnl = getattr(trade_analysis.lost, 'pnl', {}).get('total', 0)
            print(f"❌ Losing Trades: {lost_trades}")
            if lost_pnl:
                print(f"💸 Losing PnL: ${lost_pnl:.2f}")
    
    # Performance metrics
    print(f"\n=== PERFORMANCE METRICS ===")
    
    # Sharpe Ratio
    sharpe_analysis = strat.analyzers.sharpe.get_analysis()
    if sharpe_analysis and 'sharperatio' in sharpe_analysis:
        sharpe_ratio = sharpe_analysis['sharperatio'] or 0
        print(f"📊 Sharpe Ratio: {sharpe_ratio:.4f}")

    # Drawdown Analysis
    drawdown_analysis = strat.analyzers.drawdown.get_analysis()
    if drawdown_analysis:
        max_dd = drawdown_analysis.get('max', {}).get('drawdown', 0)
        print(f"📉 Max Drawdown: {max_dd:.2f}%")
    
    return results, total_trades, final_value

print("✓ Backtest function ready for Long_Candlesticks")

✓ Backtest function ready for Strategy000Simplified


In [6]:
# Execute Long_Candlesticks Backtest
try:
    results, trade_count, final_value = test_simplified_strategy()
    
    print(f"\n🎉 BACKTEST COMPLETED SUCCESSFULLY!")
    print(f"📊 Summary: {trade_count} trades, Final Value: ${final_value:,.2f}")
    
    if trade_count > 10:
        print("✅ Strategy shows active trading with multiple trades")
    elif trade_count > 1:
        print("📈 Strategy executed multiple trades")
    else:
        print("⚠️ Strategy was very selective with few trades")
        
except Exception as e:
    print(f"❌ Error running backtest: {str(e)}")
    print("💡 Make sure the S&P 500 data file exists at the specified path")

=== TESTING STRATEGY000SIMPLIFIED ===

📊 Data: 1256 trading days from 2020-09-17 to 2025-09-17
💰 Starting Capital: $100,000.00
🚀 Running backtest...

❌ Error running backtest: module 'backtrader.indicators' has no attribute 'CDLHAMMER'
💡 Make sure the S&P 500 data file exists at the specified path
